In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import dash_leaflet as dl
import plotly.express as px

from AnimalShelter import AnimalShelter  # Import CRUD python module

# Setup credentials and connect to the database
username = "aacuser"
password = "aacpass"
db = AnimalShelter(username, password)

# Initialize app
app = JupyterDash('AFillmore SNHU CS-340 Dashboard')

# Function to fetch data based on rescue type
def fetch_data(rescue_type):
    if rescue_type == 'Reset':
        return pd.DataFrame.from_records(db.read({})).drop(columns=['_id'], inplace=False)
    else:
        return pd.DataFrame.from_records(db.read_rescue_type(rescue_type)).drop(columns=['_id'], inplace=False)

# Layout of the dashboard
app.layout = html.Div([
    html.Div([
        # Logo linked to Grazioso Salvare's homepage
        html.A([
            html.Img(
                src='/assets/Grazioso_Salvare_Logo.png',
                style={'height': '100px', 'width': 'auto'}
            )
        ], href='http://www.snhu.edu', target='_blank'),
    ], style={'textAlign': 'center'}),
    html.Center(html.B(html.H1('AFillmore SNHU CS-340 Dashboard'))),
    html.Hr(),
    dcc.Dropdown(
        id='rescue-type-dropdown',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',  # Default value
        clearable=False
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[],
        data=[],
        page_size=10,
        row_selectable='single',
        style_table={'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'textAlign': 'left'
        },
    ),
    html.Br(),
    dl.Map(id='map-id', style={'width': '800px', 'height': '400px'}, center=[30.75,-97.48], zoom=9, children=[
        dl.TileLayer()
    ]),
    html.Hr(),
    dcc.Graph(id='additional-chart')
])

# Callback to update data table and set table columns based on dropdown selection
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('rescue-type-dropdown', 'value')]
)
def update_table(selected_rescue_type):
    df = fetch_data(selected_rescue_type)
    columns = [{"name": i, "id": i} for i in df.columns]
    return df.to_dict('records'), columns

# Callback to update the map based on the selected data entry
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'), Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, selected_rows):
    if viewData is None or not selected_rows:
        return [dl.TileLayer()]  # Only show base map if no data is selected
    
    dff = pd.DataFrame(viewData)
    selected_row = selected_rows[0] if selected_rows else 0
    if 'latitude' in dff.columns and 'longitude' in dff.columns:
        return [
            dl.TileLayer(),
            dl.Marker(position=[dff.at[selected_row, 'latitude'], dff.at[selected_row, 'longitude']],
                      children=[
                          dl.Tooltip(dff.at[selected_row, 'name']),
                          dl.Popup([
                              html.H1("Pet Name"),
                              html.P(dff.at[selected_row, 'name'])
                          ])
                      ])
        ]

# Callback for additional chart based on the selected data
@app.callback(
    Output('additional-chart', 'figure'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_additional_chart(data):
    if data is None:
        return px.pie(title="No data available")

    # Convert the data to a DataFrame
    dff = pd.DataFrame(data)

    preferred_breeds = [
    'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland',
    'German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler',
    'Doberman Pinscher', 'Golden Retriever', 'Bloodhound'
]

    # Filter the DataFrame to only include preferred breeds
    dff = dff[dff['breed'].isin(preferred_breeds)]

    # Check if the filtered DataFrame is empty
    if dff.empty:
        return px.pie(title="No preferred breeds available")

    # Count the number of occurrences for each preferred breed
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # Create the pie chart
    data = px.pie(breed_counts, names='breed', values='count', title='Distribution of Preferred Breeds for Selected Rescue Type')

    # Return the figure
    return data

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)
